# Data Preparation

This Jupyter Notebook cleans & prepares the raw Steam Games dataset for further analysis.

In [28]:
# Import libraries
import numpy as np
import pandas as pd
import re

from dateutil.parser import parse
from sklearn.preprocessing import MultiLabelBinarizer

In [29]:
games = pd.read_csv('datasets/raw_steam_games.csv')
games

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7,

## Removing unused rows & columns

In [30]:
games = games[games['types'] == 'app'] # remove non-game rows
games = games.dropna(subset = ['all_reviews', 'release_date']) # remove games that do not have `all_review`/`release_date`, i.e. unreleased games
games = games[~games['all_reviews'].str.contains('Need more user reviews to generate a score')] # remove games that do not have enough user reviews yet
games

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40557,https://store.steampowered.com/app/652810/Grab...,app,Grabity,"Dance the tango of death in Grabity, a fast-pa...",NaN,"Positive,(17),- 100% of the 17 user reviews fo...","May 30, 2018",Team Ninja Thumbs,"Team Ninja Thumbs,Team Ninja Thumbs","Indie,Action,4 Player Local,Local Multiplayer,...","Single-player,Multi-player,Online Multi-Player...","English,German,French,Portuguese,Spanish - Spain",15.0,"Action,Indie",About This Game Welcome to Grabity's ballis

In [31]:
games = games.drop(['url', 'types', 'desc_snippet', 'recent_reviews', 'game_description', 'game_details', 'minimum_requirements', 'recommended_requirements', 'achievements'], axis = 1) # All these fields are either not useful or too difficult to extract useful information from.
games

,name,all_reviews,release_date,developer,publisher,popular_tags,languages,genre,mature_content,original_price,discount_price
0,DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,$19.99,$14.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,$29.99,NaN
2,BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",NaN,$39.99,NaN
3,DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,$44.99,NaN
4,EVE Online,"Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...
40557,Grabity,"Positive,(17),- 100% of the 17 user reviews fo...","May 30, 2018",Team Ninja Thumbs,"Team Ninja Thumbs,Team Ninja Thumbs","Indie,Action,4 Player Local,Local Multiplayer,...","English,German,French,Portuguese,Spanish - Spain","Action,Indie",NaN,Download Demo,NaN
40598,Detached: Non-VR Edition,"Mostly Positive,(14),- 78% of the 14 user revi...","Jul 24, 2018",Anshar Studios,"Anshar Studios,Anshar Studios","Simulation,Indie,Space,Exploration,Atmospheric...","English,French,German,Simplified Chinese,Polish","Indie,Simulation",NaN,$14.99,NaN
40611,A Room Beyond,"Positive,(11),- 90% of the 11 user reviews for...","Jun 13, 2017",René Bühling,"René Bühling,René Bühling","Adventure,Indie,Point & Click","English,German,French,Italian,Spanish - Spain","Adventure,Indie",NaN,$7.99,NaN
40728,Chasing the Stars,"Mostly Positive,(10),- 70% of the 10 user revi...","Jan 23, 2019",Ertal Games,"Ertal Games,Ertal Games","Nudity,Sexual Content,Indie,Steampunk,Story Ri...",English,Indie,Mature Content Description The developers de...,$9.99,NaN


## Cleaning Data

In [32]:
def map_all_reviews(x):
    match = re.search(r"\d{1,3}%", x)
    if match:
        return match.group(0)[:-1]

def map_original_price(x):
    """
    Transforms the given price's string representation to a numerical value. Any string that is not a price, i.e. "Play for free" is converted to 0.
    :param x: a price's string representation
    :return: a numerical price
    """
    if pd.isna(x):
        return x
    elif re.match(r'(\$\d+(\.\d{2})?)', x):
        return x[1:]
    else:
        return 0.0

def one_hot_encode(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: [] if pd.isna(x) else list(x.split(',')))
    binarizer = MultiLabelBinarizer(sparse_output = True)

    tags = pd.DataFrame.sparse.from_spmatrix(binarizer.fit_transform(df.pop(column_name)), index=df.index, columns=binarizer.classes_)
    tags = tags.add_prefix(f'{column_name}_')

    df = pd.concat([df, tags], axis=1)
    return df

games['all_reviews'] = games['all_reviews'].apply(map_all_reviews)
games['release_date'] = games['release_date'].apply(lambda x: parse(x).strftime('%Y-%m-%d'))
games = one_hot_encode(games, 'popular_tags')
games = one_hot_encode(games, 'languages')
games = one_hot_encode(games, 'genre')
games['mature_content'] = games['mature_content'].apply(lambda x: not pd.isna(x))
games['original_price'] = games['original_price'].apply(map_original_price)
games['discount_price'] = games['discount_price'].apply(lambda x: x if pd.isna(x) else x[1:])
games

,name,all_reviews,release_date,developer,publisher,mature_content,original_price,discount_price,popular_tags_1980s,popular_tags_1990's,...,genre_RPG,genre_Racing,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Valve,genre_Video Production,genre_Web Publishing
0,DOOM,92,2016-05-12,id Software,"Bethesda Softworks,Bethesda Softworks",False,19.99,14.99,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PLAYERUNKNOWN'S BATTLEGROUNDS,49,2017-12-21,PUBG Corporation,"PUBG Corporation,PUBG Corporation",True,29.99,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BATTLETECH,71,2018-04-24,Harebrained Schemes,"Paradox Interactive,Paradox Interactive",False,39.99,NaN,0,0,...,0,0,0,0,0,1,0,0,0,0
3,DayZ,61,2018-12-13,Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive",False,44.99,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
4,EVE Online,74,2003-05-06,CCP,"CCP,CCP",False,0.0,NaN,0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40557,Grabity,100,2018-05-30,Team Ninja Thumbs,"Team Ninja Thumbs,Team Ninja Thumbs",False,0.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
40598,Detached: Non-VR Edition,78,2018-07-24,Anshar Studios,"Anshar Studios,Anshar Studios",False,14.99,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
40611,A Room Beyond,90,2017-06-13,René Bühling,"René Bühling,René Bühling",False,7.99,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
40728,Chasing the Stars,70,2019-01-23,Ertal Games,"Ertal Games,Ertal Games",True,9.99,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# We store the results as Pickle instead of CSV since it allows us to preserve data structures, i.e. sparse lists.
games.to_pickle('datasets/cleaned_steam_games.pkl')

# To read the file, `df = pd.read_pickle('datasets/cleaned_steam_games.pkl')`